# MUSA 650 Homework 2: Supervised Land Use Classification with Google Earth Engine

In this assignment, you will use Google Earth Engine via Python to implement multi-class land cover classification. You will hand-label Landsat 8 satellite images which you will then use to train a random forest model. Along the way, you will consider practical remote sensing issues like cloud cover, class imbalances, and feature selection.

**Given that hand-labeling data can be time-consuming, you are encouraged to work in pairs or groups of three to share the workload. You may collaborate on generating the hand-labeled data, provided you submit separate assignment files. If you choose to do this, you should all use the same ROI, of course.**

You are responsible for figuring out the code independently and may refer to tutorials, code examples, or use AI support, but **please cite all sources**.

In particular, we encourage you to consult the [official Python Google Earth Engine `geemap` package](https://geemap.org/), the online course [Spatial Thoughts](https://spatialthoughts.com/courses/google-earth-engine/), and the [Google Earth Engine Tutorials book](https://google-earth-engine.com/).

Submit a single Jupyter Notebook containing code, narrative text, visualizations, and answers to each question. Please also upload your classification results as a GeoTIFF and your accuracy assessment as a CSV file. Open a pull request from your fork of this repository to the main repository for submission.

## 1. Setup

For this assignment, you will define the region of interest (ROI) of your choice. We recommend picking an urban area large enough that you will have a sufficient sample size but not so large that it will take an excessively long time to process.

You'll also use Landsat 8 satellite imagery from USGS for this assignment. Choose images from 2023, filtering for images with minimal cloud cover.

In [82]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import geopandas as gpd
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
ee.Authenticate()
ee.Initialize(project='musa-6500-spring-2025')

In [78]:
# Region of Interest: Seattle
seattle_bbox = ee.Geometry.Rectangle([-122.435, 47.495, -122.225, 47.735])

# Define a boundary with small buffer
small_area = seattle_bbox.buffer(2000)

# Apply scaling factor to Landsat bands
def applyScaleFactor(landsat):
  opticalBands = landsat.select("SR_B.").multiply(0.0000275).add(-0.2)
  return landsat.addBands(opticalBands, None, True)

# Load Landsat 8 Collection
# Pull the image for the least cloudy day in 2023
seattle_landsat = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .filterDate("2023-01-01", "2023-12-31")  # Filter for 2023
           .filterBounds(seattle_bbox)  # Filter for Seattle area
           .map(applyScaleFactor)  # Apply scaling factor to the bands
           .sort("CLOUD_COVER")  # Sort by cloud cover percentage (least cloudy)
           .first()  # Select the least cloudy image
           .clip(seattle_bbox))  # Clip the image to the Seattle boundary

seattle_landsat_export = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .filterDate("2023-01-01", "2023-12-31")  # Filter for 2023
           .filterBounds(seattle_bbox)  # Filter for Seattle area
           .map(applyScaleFactor)  # Apply scaling factor to the bands
           .sort("CLOUD_COVER")  # Sort by cloud cover percentage (least cloudy)
           .first()  # Select the least cloudy image
           .clip(small_area))  # Clip the image to the Seattle boundary

In [79]:
# Check the date which the landsat image is pulled from
# Ensures that I've pulled values from only one day
print("The Image Date is:", ee.Date(seattle_landsat.get("system:time_start")).format("YYYY-MM-dd").getInfo())
print("The Image Date is:", ee.Date(seattle_landsat_export.get("system:time_start")).format("YYYY-MM-dd").getInfo())

The Image Date is: 2023-08-16
The Image Date is: 2023-08-16


**<u>Remark:</u>** The image that is used for labeling is actually from April 10, 2023 ("2023-04-10"), but somehow when I re-run the codes it gives me an image from August instead.

In [80]:
# Export Landsat (True Color Image 432)
# Reduce the resolution to reduce file size
geemap.ee_export_image(seattle_landsat_export, scale=60, filename="/content/drive/MyDrive/landsat_seattle_2023_v3.tif", region=small_area)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/landsat_seattle_2023_v3.tif


In [76]:
# Visualization Settings for Seattle image
visualization = {
  "bands": ["SR_B4", "SR_B3", "SR_B2"],  # True color (red, green, blue)
  "min": 0.0,
  "max": 0.3
}

In [86]:
# Create Map for visualization (using geemap or other interface)
Map = geemap.Map()
Map.centerObject(seattle_landsat, 11)  # Zoom to Seattle
Map.addLayer(seattle_landsat, visualization, "True Color (432)")
Map

Map(center=[47.61495611051143, -122.3299999999987], controls=(WidgetControl(options=['position', 'transparent_…

In [81]:
# Create Map for visualization (using geemap or other interface)
Map = geemap.Map()
Map.centerObject(seattle_landsat_export, 11)  # Zoom to Seattle
Map.addLayer(seattle_landsat_export, visualization, "True Color (432)")
Map

Map(center=[47.614952935580554, -122.33000000002237], controls=(WidgetControl(options=['position', 'transparen…

## 2. Data Collection and Feature Engineering

### 2.1 Collecting and Labeling Training Data

Using the [interactive `geemap` intereface](https://www.youtube.com/watch?v=VWh5PxXPZw0) or another approach (e.g., QGIS, ArcGIS, a GeoJSON file, etc.), create at least 100 samples (points or polygons) for each of the following four classes: urban, bare, water, and vegetation. (Again, we encourage you to work in pairs or groups of three to generate these hand labels.) Use visual cues and manual inspection to ensure that the samples are accurate. Assign each class a unique label (e.g., 0 for urban, 1 for bare, 2 for water, and 3 for vegetation) and merge the labeled samples into a single dataset. You are free to propose any labels you like, as long as 1) you include at least 4 classes, and 2) you justify why they are appropriate for a remote sensing task (for example, including a label for ice cream shops wouldn't make sense, because those can't be detected from aerial imagery).

In [84]:
gdf = gpd.read_file("/content/drive/MyDrive/labels.geojson")
gdf

,OBJECTID,Typology,geometry
0,52,W,POINT (-122.41904 47.67268)
1,53,W,POINT (-122.41322 47.67313)
2,54,W,POINT (-122.41344 47.67746)
3,55,W,POINT (-122.40963 47.67365)
4,56,W,POINT (-122.4062 47.66955)
...,...,...,...
395,448,B,POINT (-122.29152 47.65514)
396,449,B,POINT (-122.2902 47.65743)
397,450,B,POINT (-122.24716 47.68179)
398,451,B,POINT (-122.25033 47.68518)


In [85]:
gdf.groupby("Typology").size()

,0
Typology,
B,100
U,100
V,100
W,100


In [99]:
import json
# Load the GeoJSON file
geojson_file = "/content/drive/MyDrive/labels.geojson"

# Load the GeoJSON as a Python dictionary
with open(geojson_file) as f:
    geojson_data = json.load(f)

# Ensure that each feature has a valid system:index as a string and remove id if it exists
for feature in geojson_data['features']:
    # Remove 'id' if it exists to avoid conflict with 'system:index'
    if 'id' in feature['properties']:
        del feature['properties']['id']

    # Assign a unique string-based system:index to each feature
    feature['properties']['system:index'] = str(feature.get('properties', {}).get('name', feature.get('id', 'default_id')))

# Convert the modified GeoJSON back into Earth Engine FeatureCollection
training_samples = geemap.geojson_to_ee(geojson_data)

Could not convert the geojson to ee.Geometry()


Exception: Cannot specify both "id" and "system:index".

In [96]:
Map = geemap.Map()
Map.centerObject(seattle_landsat_export, 11)  # Zoom to Seattle
Map.addLayer(training_samples, {}, "True Color (432)")
Map

EEException: Feature: Value of property system:index must be a string.

In [100]:
training_samples = geemap.geojson_to_ee("/content/drive/MyDrive/labels_0743.geojson")

In [102]:
import json

# Load the GeoJSON file
with open("/content/drive/MyDrive/labels_0743.geojson", "r") as f:
    geojson_data = json.load(f)

# Convert 'id' to a string for all features
for feature in geojson_data['features']:
    feature['id'] = str(feature['id'])  # Convert 'id' to string

# Save the modified GeoJSON back
with open("/content/drive/MyDrive/labels_0743_updated.geojson", "w") as f:
    json.dump(geojson_data, f)

print("GeoJSON has been updated with string IDs!")

GeoJSON has been updated with string IDs!


In [103]:
training_samples = geemap.geojson_to_ee("/content/drive/MyDrive/labels_0743_updated.geojson")
print(training_samples.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-122.41904166799998, 47.67268282100008]}, 'id': '1', 'properties': {'OBJECTID': 1, 'OBJECTID_1': 52, 'Typology': 'W'}}


In [106]:
feature_151 = training_samples.toList(151).get(150)

# Print the feature information
print(ee.Feature(feature_151).getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-122.378952374, 47.640333602000055]}, 'id': '151', 'properties': {'OBJECTID': 151, 'OBJECTID_1': 202, 'Typology': 'U'}}


In [104]:
urban = training_samples.filter(ee.Filter.eq("Typology", "U"))

In [105]:
urban.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [-122.378952374, 47.640333602000055]},
 'id': '151',
 'properties': {'OBJECTID': 151, 'OBJECTID_1': 202, 'Typology': 'U'}}

### 2.2 Feature Engineering.

For possible use in the model, calculate and add the following spectral indices:

- **NDVI** (Normalized Difference Vegetation Index)
- **NDBI** (Normalized Difference Built-up Index)
- **MNDWI** (Modified Normalized Difference Water Index)

Additionally, add elevation and slope data from a DEM. Normalize all image bands to a 0 to 1 scale for consistent model input.

For bonus points, consider adding [kernel filters](https://google-earth-engine.com/Advanced-Image-Processing/Neighborhood-based-Image-Transformation/) (e.g., edge detection, smoothing) to see if they improve model performance.

## 3. Model Training and Evaluation

### 3.1 Model Training

Split your data into a training dataset (70%) and a validation dataset (30%). Train and evaluate a random forest model using the training set with all engineered features.

After training, analyze [variable importance scores](https://stackoverflow.com/questions/74519767/interpreting-variable-importance-from-random-forest-in-gee) to justify each feature's inclusion. Identify which features are most influential in the classification. Report the final features that you keep in your model.

### 3.2 Accuracy Assessment

Use the trained model to classify the Landsat 8 image, creating a land cover classification map with classes for urban, bare, water, and vegetation (or whatever classes you have chosen).

Using the validation data, generate a confusion matrix and calculate the overall accuracy, precision, and recall. Which classes were confused most often with each other? Why do you think this was?

Visually compare your landcover data for your ROI with the corresponding [landcover data from the European Space Agency](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200). Do your classifications agree? If not, do you notice any patterns in the types of landcover where they differ, or any particular features in the imagery that are hard for your model to recognize (e.g., sand, water, or asphalt)?

Export the classified image as a GeoTIFF and the confusion matrix and accuracy metrics to a CSV file for documentation.

## 4. Reflection Questions

What limitations did you run into when completing this assignment? What might you do differently if you repeated it, or what might you change if you had more time and/or resources?

What was the impact of feature engineering? Which layers most contributed to the model? Did you expect this? Why or why not?

Did you find it difficult to create the training data by hand? Did you notice any issues with class imbalance? If so, how might you resolve this in the future (hint: consider a different sampling technique).

Did your model perform better on one class than another? Why? Can you think of a reason that this might be good or bad depending on the context?